In [1]:
import cv2
import os
import numpy as np
import scipy.stats as ss
import torch
import time
import matplotlib.pyplot as plt
skin_mean = np.array([113.82950865, 146.5547067 , 203.94020505])
skin_var = np.array([1739.0092018, 1290.22894013, 1426.94384388])
nonSkin_mean = np.array([127.98836827, 128.85303495, 102.16741058])
nonSkin_var = np.array([4390.21815255, 4127.27179659, 4125.52420851])
skin_distbn = ss.multivariate_normal(mean = skin_mean, cov = skin_var)
nonSkin_distbn = ss.multivariate_normal(mean = nonSkin_mean, cov = nonSkin_var)

In [17]:
rootdir = "../StateFarmDistractedDriverDetection/val/c0"
imgs = []
file_list = []
for subdir, dirs, files in os.walk(rootdir):
    for f in files:
        imgs.append(cv2.imread(os.path.join(subdir, f)))
        file_list.append(f)

In [18]:
if not os.path.exists('skin_segmented_output/val/c0'):
    os.makedirs('skin_segmented_output/val/c0')

In [19]:
out_dir = 'skin_segmented_output/val/c0'

In [20]:
index = 0
while index+200 <len(imgs):
    imgs_batch = imgs[index:index+200]
    time1 = time.time()
    imgs_np = np.array(imgs_batch, dtype = np.double)
    imgs_tensor = torch.from_numpy(imgs_np)
    val1 = skin_distbn.pdf(imgs_tensor)
    val2 = skin_distbn.pdf(imgs_tensor) + nonSkin_distbn.pdf(imgs_tensor)
    val = np.divide(val1,val2)
    time2 = time.time()
    print ("Time taken is ",time2-time1,"seconds")
    
    thresh = 0.5
    bool_mat = val> thresh
    bool_mat = np.array(bool_mat,dtype = np.int8)
    mask = 1- bool_mat
    mask_3d = np.repeat(mask[:, :, :,np.newaxis], 3, axis=3)
    result = mask_3d* imgs_np + [0,255,204] * (1-mask_3d) * imgs_np
    
    for i,item in enumerate(result):
        cv2.imwrite(os.path.join(out_dir,file_list[i+index]),item)
    index+=200
    #print index

36.193862915
39.0650539398
40.1262068748
37.1978070736
36.8142678738
38.622781992
40.0171420574
38.1715779305
37.8634989262


In [21]:
imgs_batch = imgs[index:]
time1 = time.time()
imgs_np = np.array(imgs_batch, dtype = np.double)
imgs_tensor = torch.from_numpy(imgs_np)
val1 = skin_distbn.pdf(imgs_tensor)
val2 = skin_distbn.pdf(imgs_tensor) + nonSkin_distbn.pdf(imgs_tensor)
val = np.divide(val1,val2)
time2 = time.time()
print ("Time taken is ",time2-time1,"seconds")

thresh = 0.5
bool_mat = val> thresh
bool_mat = np.array(bool_mat,dtype = np.int8)
mask = 1- bool_mat
mask_3d = np.repeat(mask[:, :, :,np.newaxis], 3, axis=3)
result = mask_3d* imgs_np + [0,255,204] * (1-mask_3d) * imgs_np

for i,item in enumerate(result):
    cv2.imwrite(os.path.join(out_dir,file_list[i+index]),item)
plt.imshow(item)

22.7682979107
